In [1]:
print('LOADING A TORCHSCRIPT MODEL IN C++。。。')

LOADING A TORCHSCRIPT MODEL IN C++。。。


In [9]:
import torch
import torchvision.models as models
model = models.resnet18()
example = torch.rand(1, 3, 224, 224)
traced_script_module = torch.jit.trace(model, example)


In [10]:
output = traced_script_module(torch.ones(1, 3, 224, 224))
print(output[0, :5])

tensor([ 0.6054,  0.6028, -0.2612, -0.1908, -0.0035], grad_fn=<SliceBackward>)


In [11]:
print('Converting to  Torch Script via Annotation.')
import torch
class MyModule(torch.nn.Module):
    def __init__(self, N, M):
        super(MyModule, self).__init__()
        self.weight = torch.nn.Parameter(torch.rand(N, M))
    def forward(self, input):
        if input.sum() > 0:
            output = self.weight.mv(input)
        else:
            output = self.weight + input
        return output
my_module = MyModule(10, 20)
sm = torch.jit.script(my_module)
print(sm.code)

Converting to  Torch Script via Annotation.
def forward(self,
    input: Tensor) -> Tensor:
  _0 = torch.gt(torch.sum(input, dtype=None), 0)
  if bool(_0):
    output = torch.mv(self.weight, input)
  else:
    output = torch.add(self.weight, input, alpha=1)
  return output



In [12]:
print('Step 2: Serializing Your Script Module to a File')
traced_script_module.save('traced_resnet_model.pt')

Step 2: Serializing Your Script Module to a File


In [14]:
print('Step 3: Loading Your Script Module in C++')
traced_resnet18 = torch.jit.load('traced_resnet_model.pt')


Step 3: Loading Your Script Module in C++


In [18]:
import io
import torchvision.transforms as tfs
from PIL import Image
import json
imagenet_class_index = json.load(open(r'./imagenet_class_index.json'))
from torchvision import models
import torch
model = models.resnet18(pretrained=True)
model.eval()

def transform_image(image_bytes):
    my_transforms = tfs.Compose([
        tfs.Resize(255),
        tfs.CenterCrop(224),
        tfs.ToTensor(),
        tfs.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
        ])
    image = Image.open(io.BytesIO(image_bytes))
    return my_transforms(image).unsqueeze(0)

def get_prediction(image_bytes):
    tensor = transform_image(image_bytes=image_bytes)
    outputs = model.forward(tensor)
    _, y_hat = outputs.max(1)
    predicted_idx = str(y_hat.item())
    return imagenet_class_index[predicted_idx]

with open('../data/elep.jpg', 'rb') as f:
    image_bytes = f.read()
    print(get_prediction(image_bytes=image_bytes))


['n02504458', 'African_elephant']


In [19]:
example = torch.rand(1, 3, 224, 224)
traced_script_module = torch.jit.trace(model, example)
traced_script_module.save('traced_resnet_model.pt')

In [20]:
t_model = torch.jit.load('traced_resnet_model.pt')
with open('../data/elep.jpg', 'rb') as f:
    image_bytes = f.read()
    tensor = transform_image(image_bytes=image_bytes)
    outputs = t_model.forward(tensor)
    _, y_hat = outputs.max(1)
    predicted_idx = str(y_hat.item())
    print(imagenet_class_index[predicted_idx])


['n02504458', 'African_elephant']
